In [2]:
import pandas as pd
import numpy as np
import os

# 0. 경로 설정
file_path = 'dart_statements_merged.xlsx'
output_path = '../data/processed/dart_bs_wide.csv'

# 1. 데이터 불러오기
df = pd.read_excel(file_path, sheet_name='Sheet1')

# 2. 재무상태표(BS) 필터링
bs_df = df[df['sj_div'] == 'BS']

# 3. 주요 항목 설정
target_accounts = [
    '자산총계', '유동자산', '비유동자산',
    '부채총계', '자본총계'
]
bs_df = bs_df[bs_df['account_nm'].isin(target_accounts)]

# 4. wide-format 변환
wide_df = bs_df.pivot_table(
    index=['corp_name', 'bsns_year'],
    columns='account_nm',
    values='thstrm_amount',
    aggfunc='first'
).reset_index()

# 5. 문자열 → 숫자 변환 (콤마 제거 + '-' → NaN 처리)
for col in target_accounts:
    wide_df[col] = (
        wide_df[col]
        .replace({',': '', '-': np.nan}, regex=True)
        .astype(float)
    )

# 6. 결측값 처리 방법 선택
fillna_strategy = 'mean'  # 'zero' 또는 'mean' 중 선택

if fillna_strategy == 'zero':
    wide_df[target_accounts] = wide_df[target_accounts].fillna(0)
elif fillna_strategy == 'mean':
    wide_df[target_accounts] = wide_df[target_accounts].fillna(wide_df[target_accounts].mean())

# 7. 결과 저장
os.makedirs(os.path.dirname(output_path), exist_ok=True)
wide_df.to_csv(output_path, index=False)

# 8. 미리보기
print(wide_df.head())


account_nm corp_name  bsns_year          부채총계         비유동자산          유동자산  \
0                 3S       2015  2.942016e+10  5.129089e+10  1.606908e+10   
1                 3S       2016  2.314263e+10  4.431972e+10  1.342667e+10   
2                 3S       2017  2.452724e+10  3.699403e+10  1.671287e+10   
3                 3S       2018  1.876866e+10  3.671933e+10  1.280167e+10   
4                 3S       2019  1.895754e+10  3.524989e+10  1.807934e+10   

account_nm          자본총계          자산총계  
0           3.793981e+10  6.735997e+10  
1           3.460376e+10  5.774639e+10  
2           2.917967e+10  5.370690e+10  
3           3.075234e+10  4.952100e+10  
4           3.437168e+10  5.332922e+10  
